In [1]:
# First cell - Add this at the start of your notebook
from hypster.ui import apply_vscode_theme

apply_vscode_theme()

In [2]:
# First cell
import logging
import sys

# Set up the logger
logger = logging.getLogger("hypster.ui")  # Get the root logger for hypster
logger.setLevel(logging.DEBUG)

# Create console handler with a higher log level
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter("%(name)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# # Add the handler to the logger
logger.addHandler(console_handler)

In [3]:
from hypster import HP, config


@config
def conditional_config(hp: HP):
    model = hp.select(["cnn", "rnn", "transformer"], default="rnn", name="model")
    var = hp.multi_int([1, 2, 3])
    var2 = hp.multi_number([1.0, 2.0, 3.0])
    var3 = hp.multi_text(["a", "b"])
    if model == "cnn":
        layers = hp.select([3, 5, 7], default=5, name="layers")
        kernel_size = hp.select([3, 5], default=3, name="kernel_size")
        toggle = hp.bool(True)
    elif model == "rnn":
        cell_type = hp.select(["lstm", "gru"], default="lstm", name="cell_type")
        num_layers = hp.int(5, name="num_layers", min=1, max=100)
        if num_layers > 10:
            text = hp.text("hello")
            # if len(text) > 10:
            new = hp.multi_select(["a", "b", "c"], default=["a", "b"])
    else:  # transformer
        num_heads = hp.select([4, 8, 16], default=8, name="num_heads")
        num_layers = hp.int(2, name="num_layers", min=1, max=10)
        number = hp.number(0.01, min=0.01, max=0.1)
        number2 = hp.number(0.01)

    lr = hp.select([0.001, 0.01, 0.1], default=0.01, name="lr")

In [4]:
from hypster import HP, config


@config
def nested_config(hp: HP):
    model = hp.select(["cnn", "rnn", "transformer"], default="rnn", name="model")
    va23 = hp.multi_int([1, 2, 3])


nested_config.save("helper_configs/nested_config.py")

In [5]:
from hypster import HP, config


@config
def hp_config(hp: HP):
    lr = hp.number(0.01, min=0.01, max=0.1)
    ml = hp.propagate("helper_configs/nested_config.py")

In [6]:
from hypster import HP, config

In [7]:
@config
def fast_embed(hp: HP):
    model = hp.select(
        {"bge-small": "BAAI/bge-small-en-v1.5", "mini-lm": "sentence-transformers/all-MiniLM-L6-v2"},
        default="mini-lm",
    )
    import os

    cpu_count = os.cpu_count() or 1


fast_embed.save("helper_configs/fast_embed.py")


@config
def jina_embed(hp: HP):
    model = hp.select({"v3": "jina-embeddings-v3", "v2": "jina-embeddings-v2"}, default="v3")
    if model == "jina-embeddings-v3":
        late_chunking = hp.select([True, False], default=True, name="late_chunking")


jina_embed.save("helper_configs/jina_embed.py")

In [8]:
from hypster import HP, config


@config
def modular_rag(hp: HP):
    embedder_type = hp.select(["fastembed", "jina"], default="fastembed")
    match embedder_type:
        case "fastembed":
            embedder = hp.propagate("helper_configs/fast_embed.py")
        case "jina":
            embedder = hp.propagate("helper_configs/jina_embed.py")

In [9]:
# Test usage
from hypster.ui import interactive_config

# Initialize the interactive configuration
results = interactive_config(modular_rag)

hypster.ui.handler - DEBUG - Nested latest records: ['model']
hypster.ui.ipywidgets - DEBUG - Initialized IPyWidgetsUI
hypster.ui.ipywidgets - DEBUG - Updating display
hypster.ui.ipywidgets - DEBUG - Ordered components: ['embedder_type', 'embedder']
hypster.ui.ipywidgets - DEBUG - Processing component: embedder_type (select)
hypster.ui.ipywidgets - DEBUG - Creating new UI component for: embedder_type
hypster.ui.ipywidgets - DEBUG - Processing component: embedder (propagate)
hypster.ui.ipywidgets - DEBUG - Creating new UI component for: embedder


In [14]:
# Test usage
from hypster.ui import interactive_config

# Initialize the interactive configuration
results = interactive_config(conditional_config)

hypster.ui.handler - ERROR - Error running config: unhashable type: 'list'
hypster.ui.ipywidgets - DEBUG - Initialized IPyWidgetsUI
hypster.ui.ipywidgets - DEBUG - Updating display
hypster.ui.ipywidgets - DEBUG - Ordered components: ['model']
hypster.ui.ipywidgets - DEBUG - Processing component: model (select)
hypster.ui.ipywidgets - DEBUG - Creating new UI component for: model


hypster.ui.ipywidgets - DEBUG - Change detected - Component: model, Value: transformer, Delay: False
hypster.ui.ipywidgets - DEBUG - Handling change for model: transformer
hypster.ui.handler - DEBUG - Component ID: model
hypster.ui.handler - DEBUG - New value: transformer
hypster.ui.handler - DEBUG - Components: ['model']
hypster.ui.handler - DEBUG - Values: {'model': 'transformer'}
hypster.ui.handler - WARNING - Error running config: unhashable type: 'list'
hypster.ui.handler - DEBUG - Processing component model, with record name='model' parameter_type='select' run_id=UUID('c0d5a8a4-1c42-467b-8cac-f857f1df0326') source=<ParameterSource.UI: 'ui'> single_value=True default='rnn' value='transformer' is_reproducible=True options=['cnn', 'rnn', 'transformer'] numeric_bounds=None
hypster.ui.handler - DEBUG - Affected components: {'model': 'transformer'}
hypster.ui.ipywidgets - DEBUG - Updating UI component: model
